# Summary
경로를 따로 바꾸실 필요 없이 코드를 한 번 돌리시면 raw data(원본데이터)는 따로 복사해두고, 기존 경로("/opt/ml/input/data")의 데이터가 변경됩니다.
따라서, raw data를 다시 변경하고 싶으시다면 경로를 재설정하시기 바랍니다.

1. 원본 데이터 복사
    - "/opt/ml/input/data" 경로를 기준으로 파일을 복사하여 "/opt/ml/input/raw_data"로 저장합니다. (백업용)
2. 파일명 변경
    - "/opt/ml/input/data" 경로에 있는 데이터가 변경됩니다.
    - "change_table.csv" 파일을 기준으로 변경됩니다. 


In [126]:
import shutil, os
import pandas as pd
from itertools import chain
shutil.copytree("/opt/ml/input/data", "/opt/ml/input/test_data")



'/opt/ml/input/test_data'

In [109]:
folder_dir = '/opt/ml/input/test_data2/train/images'
folder_name_tmp = os.listdir(folder_dir)
folder_name = list(set([x.replace('._','') for x in folder_name_tmp]))  # '._' 제거
folder_name.sort()

file_list = [[folder_name[i]+"/"+file for file in os.listdir(os.path.join(folder_dir, folder_name[i])) if '._' not in file and 'ipynb' not in file] for i in range(len(folder_name))]
all_file_list = list(chain(*file_list)) # 전체 파일명 (폴더별로 가져와 2차원 -> 1차원으로 풀기)

In [124]:
df = pd.read_csv('/opt/ml/change_table.csv')
df.head()

,index,error_id,error_file_name,miss_col,before,after,type
0,148,20,incorrect,mask,incorrect,normal,file
1,106,225,incorrect,gender,female,male,all
2,107,664,incorrect,gender,female,male,all
3,121,667,incorrect,gender,female,male,all
4,122,725,incorrect,gender,female,male,all


In [122]:
def change_folder_name(df, idx):  
    id_ = df['error_id'][idx]    

    before_fold = [os.path.join(folder_dir, folder) for folder in folder_name if id_ in folder][0]
    os.rename(before_fold, before_fold.replace(df['before'][idx], df['after'][idx]))

    
def change_file_name(df, idx):
    id_ = df['error_id'][idx]
    before_ = df['before'][idx]
    after_ = df['after'][idx]
    before_fold = [os.path.join(folder_dir, file) for file in all_file_list if id_ in file and (before_ in file or after_ in file)]

    tmp_src, tmp_dst = before_fold[1].split('.')
    tmp = tmp_src+'_tmp.'+tmp_dst

    os.rename(before_fold[0], tmp)
    os.rename(before_fold[1], before_fold[0])
    os.rename(tmp, before_fold[1])


In [125]:
print("="*30)

for idx in range(len(df)):
    try:
        line = df.iloc[idx]
        if line['type'] == 'file':
            change_file_name(df, idx)
            print("change file_name - {} [{} <-> {}]!".format(df['error_id'][idx], df['before'][idx], df['after'][idx]))
        elif line['type'] == 'all':
            change_folder_name(df, idx)
            print("change folder_name - {} [{} -> {}]!".format(df['error_id'][idx], df['before'][idx], df['after'][idx]))
    except:
        print("fail - ", idx)
        
print("="*30)

change file_name - 20 [incorrect -> normal]!
change file_name - 4418 [incorrect -> normal]!
change file_name - 5227 [incorrect -> normal]!
